In [2]:
from datasets import load_dataset

ds = load_dataset("hackercupai/hackercup")
ds

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    sample: Dataset({
        features: ['name', 'year', 'round', 'statement', 'input', 'solution', 'code', 'output', 'sample_input', 'sample_output', 'images'],
        num_rows: 10
    })
    full: Dataset({
        features: ['name', 'year', 'round', 'statement', 'input', 'solution', 'code', 'output', 'sample_input', 'sample_output', 'images'],
        num_rows: 284
    })
})

In [6]:
ds['sample']['name']

['meta_game',
 'tower_rush',
 'ready_go_part_2',
 'wiki_race',
 'ready_go_part_1',
 'dim_sum_delivery',
 'cheeseburger_corollary_ch1',
 'two_apples_a_day',
 'cheeseburger_corollary_ch2',
 'road_to_nutella']

In [25]:
import io
import sys
import traceback
import contextlib
from unittest.mock import patch

def check_code_structure(extracted_code):
    # Check if the phrase "__name__ == '__main__'" is present in the code
    if "__name__ == '__main__'" not in extracted_code:
        return False, "Missing `if __name__ == '__main__':` block."
    return True, None

def run_extracted_code(extracted_code, test_input):
    # Check if the structure of the code is valid
    is_valid, error_message = check_code_structure(extracted_code)
    
    # Output and Error containers
    output = io.StringIO()
    error = None
    test_input_lines = [line.strip() for line in test_input.strip().split('\n') if line.strip()]
    
    def mock_input():
        if not test_input_lines:
            raise ValueError("Not enough input data provided")
        return test_input_lines.pop(0)

    with patch('builtins.input', mock_input), contextlib.redirect_stdout(output):
        try:
            # Compile the code object
            code_obj = compile(extracted_code, '<string>', 'exec')
            
            if is_valid:
                # If `if __name__ == '__main__'` exists, use local scope simulating standalone script behavior
                local_scope = {'__name__': '__main__'}
                exec(code_obj, local_scope)
            else:
                # Execute in the global scope if `if __name__ == '__main__'` is not present
                exec(extracted_code)

        except Exception as e:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            
            # Get the line number where the error occurred
            tb = traceback.extract_tb(exc_traceback)
            line_no = tb[-1].lineno
            
            # Get the line of code that caused the error
            code_lines = extracted_code.split('\n')
            error_line = code_lines[line_no - 1] if line_no <= len(code_lines) else "Unknown"
            
            error = f"Error occurred on line {line_no}: "
            error += f"{error_line.strip()}\n"
            error += f"Exception: {exc_type.__name__}: {str(exc_value)}\n"
            return None, error

    result = output.getvalue()
    
    # Check if the result is empty and return an appropriate error
    if not result.strip():
        return None, "Error: Code did not produce any output."

    return result, error




# Example usage
extracted_code = """
def read_int():
    return int(input())

def read_ints():
    return map(int, input().split())

def solve(s, d, k):
    print(123)

solve(3,4,5)
"""


test_input = """
7
1 1 3
0 2 4
5 5 1
0 1 1
1 1 2
97 1 99
97 1 100


"""

output, error = run_extracted_code(extracted_code, test_input)
if error:
    print("Error detected:")
    print(error)
else:
    print("Output:")
    print(output)

Output:
123



In [7]:
def check_code_structure(extracted_code):
    # Check if the phrase "__name__ == '__main__'" or '__name__ == "__main__"' is present in the code
    if "__name__ == '__main__'" not in extracted_code:
        return False, "Missing `if __name__ == '__main__':` block."
    
    return True, None

def run_extracted_code(extracted_code, test_input):
    # Check if the structure of the code is valid
    is_valid, error_message = check_code_structure(extracted_code)
    if not is_valid:
        return None, f"Code structure error: {error_message}"
    
    output = io.StringIO()
    error = None
    test_input_lines = [line.strip() for line in test_input.strip().split('\n') if line.strip()]

    def mock_input():
        if not test_input_lines:
            raise ValueError("Not enough input data provided")
        return test_input_lines.pop(0)

    local_scope = {'__name__': '__main__'}
    
    with patch('builtins.input', mock_input), contextlib.redirect_stdout(output):
        try:
            # Compile the code object
            code_obj = compile(extracted_code, '<string>', 'exec')
            
            # Execute the compiled code
            exec(code_obj, local_scope)
            
        except Exception as e:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            
            # Get the line number where the error occurred
            tb = traceback.extract_tb(exc_traceback)
            line_no = tb[-1].lineno
            
            # Get the line of code that caused the error
            code_lines = extracted_code.split('\n')
            error_line = code_lines[line_no - 1] if line_no <= len(code_lines) else "Unknown"
            
            error = f"Error occurred on line {line_no}: "
            error += f"{error_line.strip()}\n"
            error += f"Exception: {exc_type.__name__}: {str(exc_value)}\n"

    return output.getvalue(), error

# Compare the output generated by the code with the expected output
def compare_with_expected_output(generated_output, expected_output):
    # Check if either output is None
    if generated_output is None:
        return 0, ["Generated output is None"]

    generated_output_lines = generated_output.strip().splitlines()
    expected_output_lines = expected_output.strip().splitlines()
    print(expected_output_lines)
    expected_output_lines = expected_output_lines.split(":")[-1]

    total_cases = len(expected_output_lines)
    matching_cases = 0
    failed_cases = []

    for i, (generated_line, expected_line) in enumerate(zip(generated_output_lines, expected_output_lines), start=1):
        if generated_line.strip() == expected_line.strip():
            matching_cases += 1
        else:
            failed_cases.append(f"Test Case #{i}: Expected '{expected_line}' but got '{generated_line}'")

    score = (matching_cases / total_cases) * 100 if total_cases > 0 else 0
    return score, failed_cases

# Evaluate generated code on test cases
def evaluate_generated_code_on_test_cases(extracted_code, test_input, test_output):
    # Run the code and get the output
    generated_output, error = run_extracted_code(extracted_code, test_input)
    
    if generated_output is None or generated_output.strip() == "":
        return 0, error or "Error: The code ran without any problem. There's no error in parsing the input. But it produces no output. Please check the entire code again.", generated_output, []
    
    # If there's an error, return it
    if error:
        return 0, error, generated_output, []

    # Compare the generated output with expected output
    score, failed_cases = compare_with_expected_output(generated_output, test_output)
    
    if failed_cases:
        error_msg = f"Test cases failed: {failed_cases}"
        return score, error_msg, generated_output, failed_cases

    return score, error, generated_output, failed_cases


test_output = """
YES
NO
YES
NO
"""
score, error, generated_output, failed_cases = evaluate_generated_code_on_test_cases(extracted_code, test_input, test_output)


['YES', 'NO', 'YES', 'NO']


AttributeError: 'list' object has no attribute 'split'

In [46]:
generated_code = extract_python_code(code_solution['solution_code']['code'])
generated_code

'def max_k_deck_burgers(C):\n    return C // 2\n\ndef main():\n    T = int(input())\n    results = []\n    for _ in range(T):\n        C = int(input())\n        results.append(f"Case # {_ + 1}: {max_k_deck_burgers(C)}")\n    print(\'\\n\'.join(results))\n\nif __name__ == \'__main__\':\n    main()'

In [68]:
def check_code_structure(extracted_code):
    # Check if the phrase "__name__ == '__main__'" or '__name__ == "__main__"' is present in the code
    if "__name__ == '__main__'" not in extracted_code:
        return False, "Missing `if __name__ == '__main__':` block."
    
    return True, None

In [73]:
def read_data():
    return list(map(int, input().split()))

def max_k_decker_cheeseburgers(budget):
    if budget < 2:
        return 0
    dp = [float('inf')] * (budget + 1)
    dp[0] = 0
    dp[2] = 1
    for i in range(4, budget + 1, 2):
        dp[i] = min(dp[i], dp[i - 2] + 1)
    for i in range(6, budget + 1, 2):
        dp[i] = min(dp[i], dp[i - 4] + 2)
    return dp[budget]

def solve_test_case(budget):
    return max_k_decker_cheeseburgers(budget)

def generate_results(num_tests, results):
    return [f"Case #{i + 1}: {results[i]}" for i in range(min(num_tests, len(results)))]

def main():
    data = read_data()
    num_tests = data[0]
    results = []
    index = 1
    while index < len(data):
        budget = data[index]
        index += 1
        if budget >= 0:
            result = solve_test_case(budget)
            results.append(result)
        else:
            break
    print('\n'.join(generate_results(num_tests, results)))

if __name__ == '__main__':
    main()

In [72]:
check_code_structure(extracted_code)

Extracted code: 
def main():
    import sys
    input = sys.stdin.read
    data = input().split()
    num_tests = int(data[0])
    results = []
    index = 1
    for _ in range(num_tests):
        budget = int(data[index])
        index += 1
        result = calculate_max_decks(budget)
        results.append(f"Case #{len(results) + 1}: {result}")
    print('
'.join(results))

if __name__ == '__main__':
    main()



(True, None)